<a href="https://colab.research.google.com/github/manishdotel/bct-076-session/blob/main/numpyproject1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
import numpy as np

In [57]:
np.random.seed(42)
X_ROW, X_COLUMN =(2,2000)


In [58]:
X_RAW =np.random.rand(X_ROW, X_COLUMN)*100
X_RAW

array([[37.45401188, 95.07143064, 73.19939418, ...,  6.89580164,
         5.70547212, 28.21870747],
       [26.17056837, 24.69787991, 90.62545805, ..., 39.45723153,
        52.99405869, 16.13673584]])

In [59]:
Y_Raw =np.concatenate(([(X_RAW[0,:])+(X_RAW[1,:])],[(X_RAW[0,:])-(X_RAW[1,:])],np.abs([(X_RAW[0,:])+(X_RAW[1,:])])))
Y_Raw

array([[ 63.62458026, 119.76931055, 163.82485223, ...,  46.35303317,
         58.69953081,  44.35544331],
       [ 11.28344351,  70.37355073, -17.42606387, ..., -32.5614299 ,
        -47.28858658,  12.08197163],
       [ 63.62458026, 119.76931055, 163.82485223, ...,  46.35303317,
         58.69953081,  44.35544331]])

In [60]:
Y_Row, Y_Col = Y_Raw.shape
Y_Raw.shape

(3, 2000)

In [61]:
Train_rato= 0.7

In [62]:
Num_data_train = int(Train_rato*X_COLUMN)
X_RAW_Train = X_RAW[:,0:Num_data_train]
X_RAW_Test = X_RAW[:,Num_data_train:]
Y_RAW_Train = Y_Raw[:,0:Num_data_train]
Y_RAW_Test = Y_Raw[:,Num_data_train:]

In [63]:
class scalar:
  def __init__(self,mean,std):
    self.mean = mean
    self.std = std

def get_scalar(row):
  mean = np.mean(row)
  std =np.std(row)
  return scalar(mean,std)

def standarize(data,scalar):
  return(data-scalar.mean)/scalar.std
def unstandarize(data,scalar):
  return(data*(scalar.std)+scalar.mean)



In [64]:
X_scalers =[get_scalar(X_RAW_Train[row,:]) for row in range(X_ROW)]
X_train = np. array([standarize(X_RAW_Train[row,:],X_scalers[row])for row in range(X_ROW)])


In [65]:
Y_scalers =[get_scalar(Y_RAW_Train[row,:]) for row in range(Y_Row)]
Y_train = np. array([standarize(Y_RAW_Train[row,:],Y_scalers[row])for row in range(Y_Row)])


In [66]:
X_test = np. array([standarize(X_RAW_Test[row,:],X_scalers[row])for row in range(X_ROW)])
Y_test = np. array([standarize(Y_RAW_Test[row,:],Y_scalers[row])for row in range(Y_Row)])


In [67]:
print([X_train[row,:].mean() for row in range(X_ROW)])

[-1.5352798397673592e-16, -4.314009467114894e-16]


In [68]:
print([X_train[row,:].std() for row in range(X_ROW)])

[1.0, 0.9999999999999999]


In [69]:
print([Y_train[row,:].mean() for row in range(Y_Row)])
print([Y_train[row,:].std() for row in range(Y_Row)])

[2.7660413642089616e-16, -7.612957883143931e-18, 2.7660413642089616e-16]
[1.0, 1.0, 1.0]


In [70]:
class layer:
  def __init__(self,layer_index,is_output,input_dim,output_dim,activation):
    self.layer_index =layer_index
    self.is_output = is_output
    self.input_dim = input_dim
    self.output_dim = output_dim
    self.activation = activation

    if layer_index != 0:
      self.W = np.random.randn(output_dim, input_dim)* np.sqrt(2/input_dim)
      self.b = np.random.randn(output_dim, 1)* np.sqrt(2/input_dim)


In [71]:
layers_dim =  (X_ROW,4,4,Y_Row)

In [72]:
Neural_Net =[]

for layer_index in range(len(layers_dim)):
  if layer_index == 0:
    Neural_Net.append(layer(layer_index,False,0,layers_dim[layer_index],'irrelevant'))
  elif layer_index +1 == len(layers_dim):
    Neural_Net.append(layer(layer_index,True,layers_dim[layer_index-1],layers_dim[layer_index],activation='linear'))
  else:
    Neural_Net.append(layer(layer_index,False,layers_dim[layer_index-1],layers_dim[layer_index],activation='relu'))

In [73]:
pred_n_param = sum([(layers_dim[layer_index]+1)*layers_dim[layer_index+1] for layer_index in range(len(layers_dim)-1)])
act_n_param = sum([Neural_Net[layer_index].W.size+Neural_Net[layer_index].b.size for layer_index in range(1,len(layers_dim))])

print(f'predicted number of hyperparameters:{pred_n_param}')
print(f'Actual number of hyperparameters:{act_n_param}')
print(f'number of data:{X_COLUMN}')

if act_n_param >= X_COLUMN:
  raise Exception("it will overfit.")


predicted number of hyperparameters:47
Actual number of hyperparameters:47
number of data:2000


In [74]:
def activation(input_,act_function):
  if act_function == 'relu':
    return np.maximum(input_, np.zeros(input_.shape))
  elif act_function == 'linear':
    return input_
  else:
     raise Exception("it no activation function.")



In [75]:
def forward_prop(input_vec, layers_dim=layers_dim, Neural_Net=Neural_Net):
    Neural_Net[0].A = input_vec
    for layer_index in range(1, len(layers_dim)):
        Neural_Net[layer_index].Z = np.add(np.dot(Neural_Net[layer_index].W, Neural_Net[layer_index-1].A), Neural_Net[layer_index].b)
        Neural_Net[layer_index].A = activation(Neural_Net[layer_index].Z, Neural_Net[layer_index].activation)
    return Neural_Net[-1].A  # ✅ Return the final layer's output


In [76]:
forward_prop(X_train).shape == Y_train.shape

True

In [77]:
def get_loss(Y,Y_Hat, metric ='mse'):
  if metric== 'mse':
    individual_loss = 0.5 * (Y_Hat-Y)** 2
    return np.mean([np.linalg.norm(individual_loss[:, col], 2) for col in range (individual_loss.shape[1])])
  else:
    raise Exception ('Loss metric is not defined')

def get_dZ_from_loss(Y,Y_Hat, metric):
  if metric== 'mse':
    return Y_Hat - Y
  else:
    raise Exception('Loss metric is not defined')

def get_dactivation(A, act_function):
  if act_function == 'relu':
    return np.maximum(np.sign(A), np.zeros(A.shape))
  elif act_function == 'linear':
    return np.ones(A.shape)
  else:
    raise Exception('Activation function is not defined')

In [78]:
def backward_prop(Y, Y_Hat, metric='mse', layers_dim=layers_dim, Neural_Net=Neural_Net, Num_Train_Datum=Num_data_train):
    dZ = None  # Initialize dZ for backward pass
    # Iterate from output layer down to first hidden layer
    for layer_index in range(len(layers_dim) - 1, 0, -1):
        current_layer = Neural_Net[layer_index]
        if layer_index == len(layers_dim) - 1:
            dZ = get_dZ_from_loss(Y, Y_Hat, metric)
        else:
            next_layer = Neural_Net[layer_index + 1]
            dZ = np.dot(next_layer.W.T, dZ) * get_dactivation(current_layer.A, current_layer.activation)

        dW = np.dot(dZ, Neural_Net[layer_index - 1].A.T) / Num_Train_Datum
        db = np.sum(dZ, axis=1, keepdims=True) / Num_Train_Datum

        # Store the computed gradients
        current_layer.dW = dW
        current_layer.db = db


In [79]:
Learning_Rate = 0.1
max_epoch = 1000000

for epoch in range(1, max_epoch + 1):
    Y_Hat_train = forward_prop(X_train)
    backward_prop(Y_train, Y_Hat_train)

    # Update weights and biases inside the loop
    for layer_index in range(1, len(layers_dim)):
        Neural_Net[layer_index].W -= Learning_Rate * Neural_Net[layer_index].dW
        Neural_Net[layer_index].b -= Learning_Rate * Neural_Net[layer_index].db

    if epoch % 100000 == 0:
        print(f'{get_loss(Y_train, Y_Hat_train):.4f}')


0.0000
0.0000
0.0000
0.0000
0.0000
0.0000
0.0000
0.0000
0.0000
0.0000


In [80]:
print(get_loss (Y_test, forward_prop(X_test)))

1.6579289897700963e-05


In [54]:

def predict(X_Raw_Any):
  X_Any =np.array([standarize (X_Raw_Any [row, :], X_scalers[row]) for row in range(X_ROW)])
  Y_Hat= forward_prop(X_Any)
  Y_Hat_Any =np.array([unstandarize (Y_Hat[row,:], Y_scalers[row]) for row in range(Y_Row)])
  return Y_Hat_Any

In [55]:
predict(np.array([[30, 70], [70, 30], [3, 5], [888, 122]]).T)

array([[  87.33879122,   91.46450319,   20.82749952, 1225.95097435],
       [ -21.84618597,  -33.89938857, -163.81793086,  564.95434913],
       [  86.44403877,   88.98791622,  110.62303493,  156.59833413]])